<a href="https://colab.research.google.com/github/Anoshawott/Foundations-of-Business-Analytics-QBUS1040/blob/master/Iris_Flowers_Classifier_QBUS1040_Tutorial_12_30_10_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

def gram_schmidt(a):
    q = []
    for i in range(len(a)):
        #orthogonalization
        q_tilde = a[i]
        for j in range(len(q)):
            q_tilde = q_tilde - np.inner(q[j], a[i])*q[j] 
        
        #Test for dependence
        if np.linalg.norm(q_tilde) <= 1e-6: # 1e-6 = 0.000001 
            print("Vectors are linearly dependent.")
            print ("GS algorithm terminates at iteration ", i+1)
            return q
        
        #Normalization
        else:
            qi = q_tilde/np.linalg.norm(q_tilde)
            q.append(qi)
    # print("Vector are linearly independent.")
    return q

def qr_fac(a_matrix):
    A = np.transpose(a_matrix) # Gram-schmidt alg. takes ROWS as vectors
    Q = np.array(gram_schmidt(A))
    Q = np.transpose(Q) # Output of Gram-schmidt alg. also takes ROWS as vectors
    R = np.matmul(np.transpose(Q), a_matrix) # R = Q^T A
    return Q, R

def back_sub(upper_mat, b):
    n = len(b)
    x = b
    for i in reversed(range(n)): # reversed() uses the reversed roder: n-1,...,0
        for j in range(i+1, n):  # update x[i] using x[i+1],...x[n-1]
            x[i] = x[i] - upper_mat[i, j]*x[j]
        x[i] = x[i]/upper_mat[i,i]
    return x

def SLE_QR(A, b): # Solve Ax=b
    Q, R = qr_fac(A) # 1. compute A = QR
    b_tilde = np.matmul(Q.T, b) # 2. compute Q^T b
    x = back_sub(R, b_tilde) # 3. solve Rx = Q^T b using back substitution
    return x

# Iris flowers classifier

In [0]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
N = len(y)  # no. of observations
A = np.concatenate((np.ones((N,1)), X), axis = 1)
print(A[:3, :])
print('Original observed response/outcome:')
print(y)

y_vir = -np.ones(N)
for i in range(100, N):
    y_vir[i] = 1
print('Transformed observations:')
print(y_vir)

theta = SLE_QR(A, y_vir)
print('The model parameters are:', theta)

print('Predicted response')
y_confidence = np.matmul(A, theta)
y_prediction = np.zeros(N)
for i in range(N):
    if y_confidence[i] >= 0:
        y_prediction[i] = 1 # it is Iris Virginica
    else:
        y_prediction[i] = -1 # it is NOT Iris Virginica
print(y_prediction)

[[1.  5.1 3.5 1.4 0.2]
 [1.  4.9 3.  1.4 0.2]
 [1.  4.7 3.2 1.3 0.2]]
Original observed response/outcome:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Transformed observations:
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

# Multi-class classifier

In [0]:
print('Original observation')
print(y)

# transformed observation
# Iris Setosa, Iris Versicolour, Iris Virginica

y_true = -np.ones((N, 3))
for i in range(N):
    y_true[i,y[i]] = 1
# print(y_true)    

theta_Set = SLE_QR(A, y_true[:,0])
print('The model parameters for Iris Setosa are:', theta_Set)

theta_Ver = SLE_QR(A, y_true[:,1])
print('The model parameters for Iris Versicolour are:', theta_Ver)

theta_Vir = SLE_QR(A, y_true[:,2])
print('The model parameters for Iris Virginica are:', theta_Vir)

# compute the confidence
yhat_0 = np.matmul(A, theta_Set)
yhat_1 = np.matmul(A, theta_Ver)
yhat_2 = np.matmul(A, theta_Vir)

print('The predicted response')
y_pre = np.zeros(N)
for i in range(N):
    y_pre[i] = np.argmax([yhat_0[i], yhat_1[i], yhat_2[i]])
print(y_pre)

Original observation
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
The model parameters for Iris Setosa are: [-0.76355422  0.13205954  0.48569574 -0.44931423 -0.11494546]
The model parameters for Iris Versicolour are: [ 2.15411795 -0.04030737 -0.89123252  0.44133841 -0.98861319]
The model parameters for Iris Virginica are: [-2.39056373 -0.09175217  0.40553677  0.00797582  1.10355865]
The predicted response
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 2. 2. 2. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1. 2. 2. 2. 1. 1. 1. 2. 1.
 1. 1. 1. 2. 1. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1